In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import datetime
import argparse
import catboost

from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib 
from catboost import CatBoostRegressor,Pool
import os
import logging

In [2]:
parser=argparse.ArgumentParser()
parser.add_argument("-sta_id",help="id stacji pogodowej")
parser.add_argument("-asn_id",help="id czujnika airly")
parser.add_argument("-time",help="godzina")
parser.add_argument("-model_name",help="nazwa modelu")
parser.add_argument("-all_models",help="czy wszystkie modele modelu")
parser.add_argument("-cit_id",help="id miasta dla którego generujemy")
parser.add_argument("-log_file",help="ścieżka do pliku z logami")
# args = parser.parse_args()
args = parser.parse_args(args=[])

In [3]:
if args.all_models is None :
    args.all_models=1

In [4]:
if args.sta_id is None :
    args.sta_id=119

In [5]:
if args.asn_id is None :
    args.asn_id=204

In [6]:
if args.cit_id is None :
    args.cit_id=4970

In [7]:
if args.time is None :
    args.time=datetime.datetime.now().strftime("%H")

In [8]:
if args.log_file is None :
    args.log_file='D:\\polution_prediction\\logs\\polution_predict.log'

In [ ]:
if args.model_name is None :
    v_month=datetime.datetime.now().month
    if (v_month>=11 or v_month<=3):
        args.model_name='zima'
    if (v_month>=4 and v_month<=5):
        args.model_name='wiosna'    
    if (v_month>=6 and v_month<=9):
        args.model_name='lato'            
    if (v_month==10):
        args.model_name='jesien'                    

In [9]:
args.model_name='zima'

In [10]:
logger = logging.getLogger('polution_predict')
hdlr = logging.FileHandler(args.log_file)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)

In [11]:
    model_path='D:\polution_prediction\models'
    model_path=model_path+'\\'+args.model_name
    # model_file = model_path+args.model_name+"/catboost_polution_model.cmb"
    model_file_name = "\\catboost_polution_model.cmb"


In [12]:
dbuser='db_cron'
dbpass='VBruj16$aW'
try:
    conn = psycopg2.connect(dbname='weatherpg',user=dbuser,host='weatherpg.db.dv.interia.pl',password=dbpass)
except:
    print("I am unable to connect to the database")

In [ ]:
# if args.all_models =1 :
#     sel_city2_model='select cit_sta_id,cit_id,cit_asn_id,cit_name_pl from weather_owner.wh_polution_models'
#     dataset_cities=pd.read_sql_query(sel_city2_model,con=conn)

In [ ]:
def  predict_model(p_sta_id,p_cit_id,p_asn_id):
    nowDate=(datetime.datetime.today()+datetime.timedelta(days=1)).strftime ("%Y-%m-%d")
    # model_path='/home/dev/polution_predict/model/'
    model_file=model_path+f'\\{p_cit_id}'+model_file_name      
    cat_features_index = [0,1]    
    model = CatBoostRegressor(iterations=50000, learning_rate=0.01, depth=10,l2_leaf_reg=27)
    try:
        if os.path.isfile(model_file):
            model.load_model(model_file)
        else:            
            logger.warning(f'model {model_file} not exist')
            return f'model {model_file} not exist'
    except os.error:
        logger.error(f'os error {model_file}')
        return f'os error {model_file}'    
    sql_query= f'select * from weather_owner.polution_table_data_predict({args.sta_id},{args.asn_id},{args.time})'
    dataset_raw =pd.read_sql_query(sql_query,con=conn)
    dataset_raw.set_index("data")
    X =pd.DataFrame(dataset_raw, columns=['timeofday','is_weekend','windspeed6','dispel50','temp6','prev_index_avg','opady6','rhumid','wind_dir_ns','wind_dir_ew'])
    y_pred = model.predict(X.values)
    cur = conn.cursor()
    cur.callproc('weather_owner.wh_polution_prediction_set', (nowDate,args.time,p_sta_id,y_pred[0].astype(float),p_cit_id))
    conn.commit()
    cur.close()
    return 'ok'

In [ ]:
if args.all_models ==1 :
    sel_city2_model='select cit_sta_id,cit_id,cit_asn_id,cit_name_pl from weather_owner.wh_polution_models'
    dataset_cities=pd.read_sql_query(sel_city2_model,con=conn)

    for row in dataset_cities.iterrows():
        cit_sta_id=f'{row[1][0]:.0f}'
        cit_id    =f'{row[1][1]:.0f}'
        cit_asn_id=f'{row[1][2]:.0f}'        
        result=predict_model(cit_sta_id,cit_id,cit_asn_id)
        if result=='ok':
            logger.info(f'import to city {cit_id} complete')                     
else:
    predict_model(args.sta_id,args.cit_id,args.asn_id)

In [ ]:
hdlr.close

In [13]:
conn.close

<function connection.close>